In [ ]:
# | default_exp _code_generator.app_generator

In [ ]:
# | export

from typing import *
import time

from yaspin import yaspin

from fastkafka._components.logger import get_logger
from fastkafka._code_generator.helper import CustomAIChat, ValidateAndFixResponse
from fastkafka._code_generator.prompts import APP_GENERATION_PROMPT

In [ ]:
from fastkafka._components.logger import suppress_timestamps

In [ ]:
# | export

logger = get_logger(__name__)

In [ ]:
suppress_timestamps()
logger = get_logger(__name__, level=20)
logger.info("ok")

[INFO] __main__: ok


In [ ]:
# | export

ENTITY_PROMPT = """{entities}
{arguments}
"""


def _generate_entities_string(plan: Dict[str, List[Dict[str, Any]]]) -> str:
    entities = "\n".join([entity["name"] for entity in plan["entities"]])
    arguments = "\n".join(
        f"\nLet's now implement the {entity['name']} class with the following arguments:\n"
        + "\n".join(f"Argument: {k}, Type: {v}" for k, v in entity["arguments"].items())
        for entity in plan["entities"]
    )

    return ENTITY_PROMPT.format(entities=entities, arguments=arguments)

In [ ]:
fixture_plan = '''
{
    "entities": [
        {
            "name": "StoreProduct",
            "arguments": {"product_name": "str", "currency": "str", "price": "float"}
        }
    ]
}
'''

expected = '''StoreProduct

Let's now implement the StoreProduct class with the following arguments:
Argument: product_name, Type: str
Argument: currency, Type: str
Argument: price, Type: float
'''
actual = _generate_entities_string(json.loads(fixture_plan))
print(actual)
assert actual == expected

StoreProduct

Let's now implement the StoreProduct class with the following arguments:
Argument: product_name, Type: str
Argument: currency, Type: str
Argument: price, Type: float



In [ ]:
fixture_plan = """
{
    "entities": [
        {
            "name": "StoreProduct",
            "arguments": {"product_name": "str", "currency": "str", "price": "float"}
        },
        {
            "name": "SellProduct",
            "arguments": {"product_name": "str", "currency": "str"}
        }
    ]
}
"""

expected = '''StoreProduct
SellProduct

Let's now implement the StoreProduct class with the following arguments:
Argument: product_name, Type: str
Argument: currency, Type: str
Argument: price, Type: float

Let's now implement the SellProduct class with the following arguments:
Argument: product_name, Type: str
Argument: currency, Type: str
'''
actual = _generate_entities_string(json.loads(fixture_plan))
print(actual)
assert actual == expected

StoreProduct
SellProduct

Let's now implement the StoreProduct class with the following arguments:
Argument: product_name, Type: str
Argument: currency, Type: str
Argument: price, Type: float

Let's now implement the SellProduct class with the following arguments:
Argument: product_name, Type: str
Argument: currency, Type: str



In [ ]:
# | export


def _get_functions_prompt(
    functions: Dict[str, Dict[str, Union[str, List[Any]]]],
    app_name: str,
    is_producer_function: bool = False,
) -> str:
    function_messages = []
    for k, v in functions.items():
        parameters = ", ".join(
            [
                f"Parameter: {param_name}, Type: {param_type}"
                for parameter in v["parameters"]
                for param_name, param_type in parameter.items()
            ]
        )
        function_message = f"""
Now lets write the following @{app_name}.consumes functions with the following details:

Write a consumes function named "{k}" which should consume messages from the "{v['topic']}" topic and set the prefix parameter to "{v['prefix']}".
The function should take the following parameters:
{parameters}

The function should implement the following business logic:
{v['description']}"""

        if is_producer_function:
            function_message += f'\n\nAfter implementing the above logic, the function should return the {v["returns"]} object.'
            function_message = function_message.replace("consumes function", "produces function").replace("which should consume messages from the", "which should produce messages to the")

        function_messages.append(function_message)

    return "\n".join(function_messages)

In [ ]:
consumes_functions = {
    "on_change_currency": {
        "topic": "change_currency",
        "prefix": "on",
        "parameters": [{"msg": "StoreProduct"}],
        "description": "Some detailed description",
    },
    "on_sell_currency": {
        "topic": "sell_currency",
        "prefix": "on",
        "parameters": [{"msg": "StoreProduct"}],
        "description": "Some very detailed description",
    }
}

app_name = "app"

expected = """
Now lets write the following @app.consumes functions with the following details:

Write a consumes function named "on_change_currency" which should consume messages from the "change_currency" topic and set the prefix parameter to "on".
The function should take the following parameters:
Parameter: msg, Type: StoreProduct

The function should implement the following business logic:
Some detailed description

Now lets write the following @app.consumes functions with the following details:

Write a consumes function named "on_sell_currency" which should consume messages from the "sell_currency" topic and set the prefix parameter to "on".
The function should take the following parameters:
Parameter: msg, Type: StoreProduct

The function should implement the following business logic:
Some very detailed description"""

actual = _get_functions_prompt(consumes_functions, app_name)
print(actual)
assert actual == expected


Now lets write the following @app.consumes functions with the following details:

Write a consumes function named "on_change_currency" which should consume messages from the "change_currency" topic and set the prefix parameter to "on".
The function should take the following parameters:
Parameter: msg, Type: StoreProduct

The function should implement the following business logic:
Some detailed description

Now lets write the following @app.consumes functions with the following details:

Write a consumes function named "on_sell_currency" which should consume messages from the "sell_currency" topic and set the prefix parameter to "on".
The function should take the following parameters:
Parameter: msg, Type: StoreProduct

The function should implement the following business logic:
Some very detailed description


In [ ]:
produces_functions = {
    "to_change_currency": {
        "topic": "change_currency",
        "prefix": "to",
        "parameters": [{"store_product": "StoreProduct"}],
        "description": "Some detailed description",
        "returns": "StoreProduct",
    }
}

app_name = "app"

expected = """
Now lets write the following @app.produces functions with the following details:

Write a produces function named "to_change_currency" which should produce messages to the "change_currency" topic and set the prefix parameter to "to".
The function should take the following parameters:
Parameter: store_product, Type: StoreProduct

The function should implement the following business logic:
Some detailed description

After implementing the above logic, the function should return the StoreProduct object."""

actual = _get_functions_prompt(produces_functions, app_name, True)
print(actual)
assert actual == expected


Now lets write the following @app.produces functions with the following details:

Write a produces function named "to_change_currency" which should produce messages to the "change_currency" topic and set the prefix parameter to "to".
The function should take the following parameters:
Parameter: store_product, Type: StoreProduct

The function should implement the following business logic:
Some detailed description

After implementing the above logic, the function should return the StoreProduct object.


In [ ]:
produces_functions = {
    "to_change_currency": {
        "topic": "change_currency",
        "prefix": "to",
        "parameters": [{"store_product": "StoreProduct"}],
        "description": "Some detailed description",
        "returns": "StoreProduct",
    },
    "to_calculate_amount": {
        "topic": "change_currency",
        "prefix": "to",
        "parameters": [{"store_product": "StoreProduct"}],
        "description": "Some detailed description",
        "returns": "StoreProduct",
    }
}

app_name = "app"

expected = """
Now lets write the following @app.produces functions with the following details:

Write a produces function named "to_change_currency" which should produce messages to the "change_currency" topic and set the prefix parameter to "to".
The function should take the following parameters:
Parameter: store_product, Type: StoreProduct

The function should implement the following business logic:
Some detailed description

After implementing the above logic, the function should return the StoreProduct object.

Now lets write the following @app.produces functions with the following details:

Write a produces function named "to_calculate_amount" which should produce messages to the "change_currency" topic and set the prefix parameter to "to".
The function should take the following parameters:
Parameter: store_product, Type: StoreProduct

The function should implement the following business logic:
Some detailed description

After implementing the above logic, the function should return the StoreProduct object."""

actual = _get_functions_prompt(produces_functions, app_name, True)
print(actual)
assert actual == expected


Now lets write the following @app.produces functions with the following details:

Write a produces function named "to_change_currency" which should produce messages to the "change_currency" topic and set the prefix parameter to "to".
The function should take the following parameters:
Parameter: store_product, Type: StoreProduct

The function should implement the following business logic:
Some detailed description

After implementing the above logic, the function should return the StoreProduct object.

Now lets write the following @app.produces functions with the following details:

Write a produces function named "to_calculate_amount" which should produce messages to the "change_currency" topic and set the prefix parameter to "to".
The function should take the following parameters:
Parameter: store_product, Type: StoreProduct

The function should implement the following business logic:
Some detailed description

After implementing the above logic, the function should return the StoreP

In [ ]:
# | export


def _generate_apps_prompt(plan: Dict[str, List[Dict[str, Any]]]) -> str:
    apps_prompt = ""
    for app in plan["apps"]:
        apps_prompt += f"""Now, lets create a instance of the FastKafka app with the following fields and assign it to the variable named {app['app_name']}:

kafka_brokers: {app["kafka_brokers"]}
title: {app["title"]}
{_get_functions_prompt(app["produces_functions"], app["app_name"], True)}
{_get_functions_prompt(app["consumes_functions"], app["app_name"])}

"""
    return apps_prompt

In [ ]:
fixture_plan = '''
{
    "apps": [
        {
            "app_name": "app",
            "kafka_brokers": "None",
            "title": "FastKafka App",
            "consumes_functions": {
                "on_store_product": {
                    "topic": "store_product",
                    "prefix": "on",
                    "parameters": [{"msg": "StoreProduct"}],
                    "description": "Some detailed description"
                }
            },
            "produces_functions": {
                "to_change_currency": {
                    "topic": "change_currency",
                    "prefix": "to",
                    "parameters": [{"store_product": "StoreProduct"}],
                    "description": "Some detailed description",
                    "returns": "StoreProduct"
                }
            }
        }
    ]
}
'''

expected = '''Now, lets create a instance of the FastKafka app with the following fields and assign it to the variable named app:

kafka_brokers: None
title: FastKafka App

Now lets write the following @app.produces functions with the following details:

Write a produces function named "to_change_currency" which should produce messages to the "change_currency" topic and set the prefix parameter to "to".
The function should take the following parameters:
Parameter: store_product, Type: StoreProduct

The function should implement the following business logic:
Some detailed description

After implementing the above logic, the function should return the StoreProduct object.

Now lets write the following @app.consumes functions with the following details:

Write a consumes function named "on_store_product" which should consume messages from the "store_product" topic and set the prefix parameter to "on".
The function should take the following parameters:
Parameter: msg, Type: StoreProduct

The function should implement the following business logic:
Some detailed description

'''
actual = _generate_apps_prompt(json.loads(fixture_plan))
print(actual)
assert actual == expected

Now, lets create a instance of the FastKafka app with the following fields and assign it to the variable named app:

kafka_brokers: None
title: FastKafka App

Now lets write the following @app.produces functions with the following details:

Write a produces function named "to_change_currency" which should produce messages to the "change_currency" topic and set the prefix parameter to "to".
The function should take the following parameters:
Parameter: store_product, Type: StoreProduct

The function should implement the following business logic:
Some detailed description

After implementing the above logic, the function should return the StoreProduct object.

Now lets write the following @app.consumes functions with the following details:

Write a consumes function named "on_store_product" which should consume messages from the "store_product" topic and set the prefix parameter to "on".
The function should take the following parameters:
Parameter: msg, Type: StoreProduct

The function sh

In [ ]:
fixture_plan = '''
{
    "apps": [
        {
            "app_name": "app_1",
            "kafka_brokers": "None",
            "title": "FastKafka 1 App",
            "consumes_functions": {
                "on_store_product": {
                    "topic": "store_product",
                    "prefix": "on",
                    "parameters": [{"msg": "StoreProduct"}],
                    "description": "Some detailed description"
                }
            },
            "produces_functions": {}
        },
        {
            "app_name": "app_2",
            "kafka_brokers": "None",
            "title": "FastKafka 2 App",
            "consumes_functions": {},
            "produces_functions": {
                "to_change_currency": {
                    "topic": "change_currency",
                    "prefix": "to",
                    "parameters": [{"store_product": "StoreProduct"}],
                    "description": "Some detailed description",
                    "returns": "StoreProduct"
                }
            }
        }
        
    ]
}
'''

expected = '''Now, lets create a instance of the FastKafka app with the following fields and assign it to the variable named app_1:

kafka_brokers: None
title: FastKafka 1 App


Now lets write the following @app_1.consumes functions with the following details:

Write a consumes function named "on_store_product" which should consume messages from the "store_product" topic and set the prefix parameter to "on".
The function should take the following parameters:
Parameter: msg, Type: StoreProduct

The function should implement the following business logic:
Some detailed description

Now, lets create a instance of the FastKafka app with the following fields and assign it to the variable named app_2:

kafka_brokers: None
title: FastKafka 2 App

Now lets write the following @app_2.produces functions with the following details:

Write a produces function named "to_change_currency" which should produce messages to the "change_currency" topic and set the prefix parameter to "to".
The function should take the following parameters:
Parameter: store_product, Type: StoreProduct

The function should implement the following business logic:
Some detailed description

After implementing the above logic, the function should return the StoreProduct object.


'''
actual = _generate_apps_prompt(json.loads(fixture_plan))
print(actual)
assert actual == expected

Now, lets create a instance of the FastKafka app with the following fields and assign it to the variable named app_1:

kafka_brokers: None
title: FastKafka 1 App


Now lets write the following @app_1.consumes functions with the following details:

Write a consumes function named "on_store_product" which should consume messages from the "store_product" topic and set the prefix parameter to "on".
The function should take the following parameters:
Parameter: msg, Type: StoreProduct

The function should implement the following business logic:
Some detailed description

Now, lets create a instance of the FastKafka app with the following fields and assign it to the variable named app_2:

kafka_brokers: None
title: FastKafka 2 App

Now lets write the following @app_2.produces functions with the following details:

Write a produces function named "to_change_currency" which should produce messages to the "change_currency" topic and set the prefix parameter to "to".
The function should take the 

In [ ]:
# | export


def _generate_app_prompt(plan: str) -> str:
    plan_dict = json.loads(plan)
    entities_prompt = _generate_entities_string(plan_dict)
    apps_prompt = _generate_apps_prompt(plan_dict)
    generated_plan_prompt = entities_prompt + "\n\n" + apps_prompt
    return APP_GENERATION_PROMPT.format(generated_plan_prompt=generated_plan_prompt)

In [ ]:
fixture_plan = """
{
    "entities": [
        {
            "name": "StoreProduct",
            "arguments": {"product_name": "str", "currency": "str", "price": "float"}
        }
    ],
    "apps": [
        {
            "app_name": "app",
            "kafka_brokers": "None",
            "title": "FastKafka App",
            "consumes_functions": {
                "on_store_product": {
                    "topic": "store_product",
                    "prefix": "on",
                    "parameters": [{"msg": "StoreProduct"}],
                    "description": "Some detailed description"
                }
            },
            "produces_functions": {
                "to_change_currency": {
                    "topic": "change_currency",
                    "prefix": "to",
                    "parameters": [{"store_product": "StoreProduct"}],
                    "description": "Some detailed description",
                    "returns": "StoreProduct"
                }
            }
        }
    ]
}
"""
generated_plan_prompt = """StoreProduct

Let's now implement the StoreProduct class with the following arguments:
Argument: product_name, Type: str
Argument: currency, Type: str
Argument: price, Type: float


Now, lets create a instance of the FastKafka app with the following fields and assign it to the variable named app:

kafka_brokers: None
title: FastKafka App

Now lets write the following @app.produces functions with the following details:

Write a produces function named "to_change_currency" which should produce messages to the "change_currency" topic and set the prefix parameter to "to".
The function should take the following parameters:
Parameter: store_product, Type: StoreProduct

The function should implement the following business logic:
Some detailed description

After implementing the above logic, the function should return the StoreProduct object.

Now lets write the following @app.consumes functions with the following details:

Write a consumes function named "on_store_product" which should consume messages from the "store_product" topic and set the prefix parameter to "on".
The function should take the following parameters:
Parameter: msg, Type: StoreProduct

The function should implement the following business logic:
Some detailed description

"""

expected = APP_GENERATION_PROMPT.format(generated_plan_prompt=generated_plan_prompt)
actual = _generate_app_prompt(fixture_plan)
print(actual)
assert actual == expected


Strictly follow the below steps while generating the Python script

==== Step by Step instruction: ==== 

We are implementing a FastKafka app (check above for description).

This app has the following Message classes:

StoreProduct

Let's now implement the StoreProduct class with the following arguments:
Argument: product_name, Type: str
Argument: currency, Type: str
Argument: price, Type: float


Now, lets create a instance of the FastKafka app with the following fields and assign it to the variable named app:

kafka_brokers: None
title: FastKafka App

Now lets write the following @app.produces functions with the following details:

Write a produces function named "to_change_currency" which should produce messages to the "change_currency" topic and set the prefix parameter to "to".
The function should take the following parameters:
Parameter: store_product, Type: StoreProduct

The function should implement the following business logic:
Some detailed description

After implementing th

In [ ]:
# | export

def _validate_response(response: str) -> str:
    # todo:
    return []

In [ ]:
# kafka_brokers are invalid
# consumes has no implementation

invalid_response = """
from typing import *
from pydantic import BaseModel, Field
from fastkafka import FastKafka


class StoreProduct(BaseModel):
    product_name: str = Field(..., description="Name of the product")
    currency: str = Field(..., description="Currency")
    price: float = Field(..., description="Price of the product")


app = FastKafka(
    title="FastKafka App",
    kafka_brokers={"localhost": {"url": "localhost", "description": "Localhost broker"}},
)


@app.produces(topic="change_currency", prefix="to")
async def to_change_currency(store_product: StoreProduct) -> StoreProduct:
    if store_product.currency == "HRK":
        store_product.currency = "EUR"
        store_product.price /= 7.5
    return store_product


@app.consumes(topic="store_product", prefix="on")
async def on_store_product(msg: StoreProduct):
    pass
"""

In [ ]:
# | export

def generate_app(plan: str, description: str) -> Tuple[str, str]:
    """Generate code for the new FastKafka app from the validated plan
    
    Args:
        plan: The validated application plan generated from the user's application description
        description: Validated user's application description
    Returns:
        The generated FastKafka code
    """
    # TODO: Generate code form the plan prompt
    # TODO: Validate the generated code
    with yaspin(text="Generating FastKafka app...", color="cyan", spinner="clock") as sp:
        app_prompt = _generate_app_prompt(plan)
        
        app_generator = CustomAIChat(user_prompt=app_prompt)
        app_validator = ValidateAndFixResponse(app_generator, _validate_response)
        validated_app, total_tokens = app_validator.fix(description)
        
        sp.text = ""
        sp.ok(" ✔ FastKafka app generated and saved at: /some_dir/application.py")
        return validated_app, total_tokens

In [ ]:
fixture_plan = '''
{
    "entities": [
        {
            "name": "StoreProduct",
            "arguments": {"product_name": "str", "currency": "str", "price": "float"}
        }
    ],
    "apps": [
        {
            "app_name": "app",
            "kafka_brokers": "None",
            "title": "FastKafka App",
            "consumes_functions": {
                "on_store_product": {
                    "topic": "store_product",
                    "prefix": "on",
                    "parameters": [{"msg": "StoreProduct"}],
                    "description": "Some detailed description"
                }
            },
            "produces_functions": {
                "to_change_currency": {
                    "topic": "change_currency",
                    "prefix": "to",
                    "parameters": [{"store_product": "StoreProduct"}],
                    "description": "Some detailed description",
                    "returns": "StoreProduct"
                }
            }
        }
    ]
}
'''

app_description = """
Create FastKafka application which consumes messages from the store_product topic, it consumes messages with three attributes: product_name, currency and price. While consuming, it should produce a message to the change_currency topic. input parameters for this producing function should be store_product object and function should return store_product. produces function should check if the currency in the input store_product parameter is "HRK", currency should be set to "EUR" and the price should be divided with 7.5.
app should use localhost broker
"""

code = generate_app(fixture_plan, app_description)
print(code)

 ✔ FastKafka app generated and saved at: /some_dir/application.py 
('import asyncio\nfrom typing import *\nfrom pydantic import BaseModel, Field, NonNegativeFloat\nfrom fastkafka import FastKafka\n\nclass StoreProduct(BaseModel):\n    product_name: str = Field(..., description="Name of the product")\n    currency: str = Field(..., description="Currency")\n    price: NonNegativeFloat = Field(..., description="Price of the product")\n\nkafka_brokers = {\n    "localhost": {\n        "url": "localhost",\n        "description": "local development kafka broker",\n        "port": 9092,\n    }\n}\n\napp = FastKafka(\n    title="FastKafka App",\n    kafka_brokers=kafka_brokers,\n)\n\n@app.produces(topic="change_currency", prefix="to")\nasync def to_change_currency(store_product: StoreProduct) -> StoreProduct:\n    if store_product.currency == "HRK":\n        store_product.currency = "EUR"\n        store_product.price /= 7.5\n    return store_product\n\n@app.consumes(topic="store_product", prefi